#### Task 1
Create one pandas dataframe that combines all the data scraped from May 22, 2022 together. Drop rows with missing job titles and/or job descriptions. Use `spacy` to tokenize all the job descriptions included in the cleaned dataframe. Count how many unique tokens are there in total, and describe the distribution of token tags among them.

In [1]:
import os
import spacy
import glob
import pandas as pd
from tqdm import tqdm

nlp = spacy.load('en_core_web_sm')
os.chdir('/Users/[editted]/Desktop/compsoc/data/indeed_scraped_data/job_info_data')

In [2]:
#Combine data from May 22 and drop rows with missing values
May22_all = pd.DataFrame()
for file in glob.glob('*.csv'):
    if file[9:16] == '5222022':
        #print(file)
        read_file = pd.read_csv(file)
        May22_all = May22_all.append(read_file, ignore_index=True)
# May22_all
cleaned_May22 = May22_all.dropna(subset=['lnks_job_title', 'lnks_job_description'])
#cleaned_May22
cleaned_May22.to_csv('May22.csv', index=False)

/var/folders/ml/2z7dml2d3vn2rqmr8h2sw_n40000gn/T/ipykernel_2648/351113638.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  May22_all = May22_all.append(read_file, ignore_index=True)
/var/folders/ml/2z7dml2d3vn2rqmr8h2sw_n40000gn/T/ipykernel_2648/351113638.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  May22_all = May22_all.append(read_file, ignore_index=True)
/var/folders/ml/2z7dml2d3vn2rqmr8h2sw_n40000gn/T/ipykernel_2648/351113638.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  May22_all = May22_all.append(read_file, ignore_index=True)
/var/folders/ml/2z7dml2d3vn2rqmr8h2sw_n40000gn/T/ipykernel_2648/351113638.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future v

In [3]:
#Get the job descriptions and tokenize them
May22 = pd.read_csv('May22.csv', low_memory=False)
print('')
describe_token = {}
for i in tqdm(range(len(May22))):
    job_description = May22.loc[i, 'lnks_job_description']
    for token in nlp(job_description):
        describe_token[token.text] = token.pos_
    token_df = pd.DataFrame(columns=['token', 'tag'])
    token_df['token'] = describe_token.keys()
    token_df['tag'] = describe_token.values()
# token_df

print('number of unique tokens in the job description:', len(describe_token))
print('the distribution of token tas among them:')
print(token_df['tag'].value_counts())

100%|██████████| 4803/4803 [19:20<00:00,  4.14it/s]  

number of unique tokens in the job description: 39150
the distribution of token tas among them:
PROPN    15204
NOUN     11208
VERB      4613
ADJ       3267
NUM       3206
ADV        819
X          198
ADP        142
PRON       112
PUNCT       83
SPACE       77
AUX         65
SCONJ       48
DET         45
INTJ        22
CCONJ       18
PART        14
SYM          9
Name: tag, dtype: int64


In [4]:
#Double Check
# unique_tokens =set()
# for i in range(len(May22)):
#     job_description = May22.loc[i, 'lnks_job_description']
#     for token in nlp(job_description):
#         unique_tokens.add(token.text)
#
# print('number of unique tokens in the job description:', len(unique_tokens))

number of unique tokens in the job description: 39150


**Task 1 Conclusion**
There are 39150 unique tokens in total. The distribution of token tags are printed as shown above.

#### Task 2
Construct a token/term frequency dictionary from the cleaned dataframe. The keys of the dictionary should be unique tokens, and the values being the frequency of the tokens in the entire corpus. Save that dictionary to a json file. What are the top 10 most common tokens according to your frequency dictionary?



In [4]:
from collections import Counter
import json

May22 = pd.read_csv('May22.csv', low_memory=False)
print('')
counts = Counter()
for i in tqdm(range(len(May22))):
    job_description = May22.loc[i, 'lnks_job_description']
    for token in nlp(job_description):
        counts[token.orth_] += 1
token_freq = dict(counts)
# token_freq
with open ('522_token_frequency.json','w') as g:
    json.dump(token_freq, g)

100%|██████████| 4803/4803 [13:10<00:00,  6.08it/s]


In [5]:
#Get the first ten rows
freq_df = pd.DataFrame(token_freq.items())
freq_df = freq_df.rename(columns={0: 'token'})
freq_df = freq_df.rename(columns={1: 'frequency'})
freq_df = freq_df.sort_values(by='frequency', ascending=False, ignore_index=True)
result = freq_df.head(10)
print(result)

  token  frequency
0     ,     254381
1   and     240686
2    \n     227694
3     .     172173
4    to     119440
5   the     100408
6    of      97891
7    in      60798
8  with      50655
9     a      49613


**Task 2 Conclusion**
According to the frequency dictionary (visualized by converting to dataframe), the 10 most common tokens in the entire corpus are: ",", "and", "\n", ".", "to", "the", "of", "in", "with" and "a".